In [21]:
# Подключаем необходимые библиотеки
import pandas as pd
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
df_financial = pd.read_csv('financial_data.csv')
df_prolong = pd.read_csv('prolongations.csv')

In [4]:
def parse_number(x):
    if isinstance(x, str):
        x = x.replace(' ', '').replace(',', '.')
        try:
            return float(x)
        except:
            return 0
    elif pd.isna(x):
        return 0
    return x

In [6]:
# Очистка числовых столбцов
month_cols = [col for col in df_financial.columns if re.match(r'[А-Яа-я]+\s\d{4}', col)]
for col in month_cols:
    df_financial[col] = df_financial[col].apply(parse_number)

merged = df_prolong.merge(df_financial, on='id', how='left')

In [15]:
months = [
    'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь',
    'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'
]
month_name_to_number = {name.lower(): i + 1 for i, name in enumerate(months)}
month_number_to_russian = {i + 1: name for i, name in enumerate(months)}

In [18]:
def calculate_coefficients(df, current_month):
    coefficients = []

    try:
        # Преобразуем строку и парсим
        month_str, year_str = current_month.strip().split()
        month_str = month_str.lower()  # приводим к нижнему регистру
        current_dt = datetime(int(year_str), month_name_to_number[month_str], 1)
    except Exception as e:
        print(f"Ошибка парсинга месяца {current_month}: {e}")
        return pd.DataFrame()


    # Месяцы для расчётов
    k1_base_dt = current_dt - relativedelta(months=1)  # для K1 – апрель
    k1_target_dt = current_dt                         # для K1 – май

    k2_base_dt = current_dt - relativedelta(months=2)  # для K2 – март
    k2_exclude_dt = current_dt - relativedelta(months=1)  # исключить апрель
    k2_target_dt = current_dt  # для K2 – май

    # Названия столбцов
    k1_base_col = f"{month_number_to_russian[k1_base_dt.month]} {k1_base_dt.year}"
    k1_target_col = f"{month_number_to_russian[k1_target_dt.month]} {k1_target_dt.year}"
    k2_base_col = f"{month_number_to_russian[k2_base_dt.month]} {k2_base_dt.year}"
    k2_exclude_col = f"{month_number_to_russian[k2_exclude_dt.month]} {k2_exclude_dt.year}"
    k2_target_col = f"{month_number_to_russian[k2_target_dt.month]} {k2_target_dt.year}"

    for manager, group in df.groupby('AM'):
        # --- K1: отгрузка за апрель и за май (для завершившихся в апреле)
        if k1_base_col in group.columns and k1_target_col in group.columns:
            base_k1 = group[k1_base_col].sum()
            prolong_k1 = group[group[k1_target_col] > 0][k1_target_col].sum()
            k1 = round(prolong_k1 / base_k1, 2) if base_k1 else None
        else:
            k1 = None

        # --- K2: отгрузка за март, нет в апреле, но есть в мае
        if k2_base_col in group.columns and k2_target_col in group.columns and k2_exclude_col in group.columns:
            filtered = group[(group[k2_exclude_col] == 0) & (group[k2_target_col] > 0)]
            base_k2 = group[(group[k2_exclude_col] == 0)][k2_base_col].sum()
            prolong_k2 = filtered[k2_target_col].sum()
            k2 = round(prolong_k2 / base_k2, 2) if base_k2 else None
        else:
            k2 = None

        coefficients.append({
            'Месяц': current_month,
            'Менеджер': manager,
            'K1': k1,
            'K2': k2
        })

    return pd.DataFrame(coefficients)

In [19]:
# Список месяцев
months_to_analyze = sorted(set(df_prolong['month']))

# Расчёт коэффициентов
results = []
for month in months_to_analyze:
    coeffs_df = calculate_coefficients(merged, month)
    if not coeffs_df.empty:
        results.extend(coeffs_df.to_dict(orient='records'))

In [20]:
final_df = pd.DataFrame(results)

# Группировки
final_df['Год'] = final_df['Месяц'].str.extract(r'(\d{4})')
yearly_report = final_df.groupby(['Год', 'Менеджер'])[['K1', 'K2']].mean().reset_index()
overall_report = final_df.groupby('Менеджер')[['K1', 'K2']].mean().reset_index()

# Сохранение
final_df.to_csv('prolongation_report_fixed.csv', index=False)
yearly_report.to_csv('yearly_report.csv', index=False)
overall_report.to_csv('overall_report.csv', index=False)

print("Все отчёты успешно сформированы и сохранены в CSV!")
print("Пример данных:")
print(final_df.head())

Все отчёты успешно сформированы и сохранены в CSV!
Пример данных:
         Месяц                      Менеджер  K1  K2   Год
0  август 2023  Васильев Артем Александрович NaN NaN  2023
1  август 2023       Иванова Мария Сергеевна NaN NaN  2023
2  август 2023      Кузнецов Михаил Иванович NaN NaN  2023
3  август 2023     Михайлов Андрей Сергеевич NaN NaN  2023
4  август 2023       Петрова Анна Дмитриевна NaN NaN  2023
